# Market Basket Analysis

### Prepare csv file

In [1]:
import pandas as pd
import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)
from scipy import stats
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.externals import joblib
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [90]:
df_original = pd.read_excel('dummy original.xlsx', sheet_name='MARKET BASKET')

In [91]:
print(df_original['CARDNUM'].values.reshape(-1).tolist())

[3, 6, 2, 5, 10, 8, 3, 8, 8, 1, 8, 3, 4, 1, 8, 6, 3, 5, 9, 8, 3, 5, 7, 6, 4, 8, 3, 9, 7, 1, 2, 1, 8, 5, 9, 2, 4, 10, 5, 6, 2, 6, 5, 3, 1, 1, 7, 5, 1, 7, 8, 10, 7, 8, 3, 1, 6, 4, 9, 10, 9, 5, 3, 4, 7, 8, 10, 8, 9, 3, 10, 4, 2, 2, 5, 1, 8, 10, 3, 4, 2, 8, 7, 5, 4, 3, 7, 6, 7, 4, 2, 7, 1, 6, 1, 1, 6, 10, 8, 1]


In [92]:
df = df_original

In [93]:
df.head(10)

,Obs,Transaction_ID,CARDNUM,Amount,TIMESTAMP,ISSR_BUS_NM,prch_dt,CPD_Month,_NAME_,PROD1,...,PROD50,PROD51,PROD52,PROD53,PROD54,PROD55,PROD56,PROD57,PROD58,PROD59
0,1,TM1,3,320000,2019-05-28 15:13:23.002,A,20190528,201905,Product_Group_Name___Level_4,1504:\nCHEESE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,TM2,6,81000,2019-04-17 16:30:48.038,B,20190419,201904,Product_Group_Name___Level_4,1424:\nCHOCOLATE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,TM3,2,226804,2019-04-26 20:05:33.043,C,20190330,201904,Product_Group_Name___Level_4,1222: Sanitary,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,TM4,5,270800,2019-03-23 16:19:13.987,C,20190309,201903,Product_Group_Name___Level_4,1440: BABY\nMILK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,TM5,10,514080,2019-03-24 13:02:16.022,D,20190511,201905,Product_Group_Name___Level_4,1440: BABY\nMILK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,TM6,8,258840,2019-04-14 14:24:06.998,C,20180728,201807,Product_Group_Name___Level_4,1440: BABY\nMILK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7,TM7,3,258840,2019-04-14 14:24:06.998,B,20180914,201809,Product_Group_Name___Level_4,1440: BABY\nMILK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,8,TM8,8,517680,2019-04-17 10:18:16.013,C,20190501,201905,Product_Group_Name___Level_4,1440: BABY\nMILK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,9,TM9,8,243720,2019-04-17 15:19:49.037,A,20190430,201905,Product_Group_Name___Level_4,1440: BABY\nMILK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10,TM10,1,243720,2019-04-17 15:19:49.037,C,20190523,201905,Product_Group_Name___Level_4,1440: BABY\nMILK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [94]:
df.shape

(100, 68)

In [95]:
column_name = df.columns

In [96]:
column_name

Index(['Obs', 'Transaction_ID', 'CARDNUM', 'Amount', 'TIMESTAMP',
       'ISSR_BUS_NM', 'prch_dt', 'CPD_Month', '_NAME_', 'PROD1', 'PROD2',
       'PROD3', 'PROD4', 'PROD5', 'PROD6', 'PROD7', 'PROD8', 'PROD9', 'PROD10',
       'PROD11', 'PROD12', 'PROD13', 'PROD14', 'PROD15', 'PROD16', 'PROD17',
       'PROD18', 'PROD19', 'PROD20', 'PROD21', 'PROD22', 'PROD23', 'PROD24',
       'PROD25', 'PROD26', 'PROD27', 'PROD28', 'PROD29', 'PROD30', 'PROD31',
       'PROD32', 'PROD33', 'PROD34', 'PROD35', 'PROD36', 'PROD37', 'PROD38',
       'PROD39', 'PROD40', 'PROD41', 'PROD42', 'PROD43', 'PROD44', 'PROD45',
       'PROD46', 'PROD47', 'PROD48', 'PROD49', 'PROD50', 'PROD51', 'PROD52',
       'PROD53', 'PROD54', 'PROD55', 'PROD56', 'PROD57', 'PROD58', 'PROD59'],
      dtype='object')

In [97]:
column_drop = ['Obs', 'Transaction_ID', 'Amount', 'TIMESTAMP',
       'ISSR_BUS_NM', 'prch_dt', 'CPD_Month', '_NAME_']

In [98]:
df.drop(column_drop, axis=1, inplace=True)

In [99]:
df.head(10)

,CARDNUM,PROD1,PROD2,PROD3,PROD4,PROD5,PROD6,PROD7,PROD8,PROD9,...,PROD50,PROD51,PROD52,PROD53,PROD54,PROD55,PROD56,PROD57,PROD58,PROD59
0,3,1504:\nCHEESE,2201:\nSAYUR TAN,1003: FRUIT\nJUI,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6,1424:\nCHOCOLATE,1008:\nWATER,1500: FRESH\nMIL,1453:\nINSTANT D,1421:\nSOFT\nCAND,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1222: Sanitary,1502: DAIRY\nDES,1414: ROLLS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5,1440: BABY\nMILK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10,1440: BABY\nMILK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,8,1440: BABY\nMILK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,3,1440: BABY\nMILK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,8,1440: BABY\nMILK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,1440: BABY\nMILK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1,1440: BABY\nMILK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [100]:
df_result_predict = pd.read_csv('result_predict.csv')

In [101]:
df_result_predict.head(10)

,cardno,train,test
0,1,NaN,5
1,2,NaN,1
2,3,NaN,1
3,4,NaN,4
4,5,NaN,1
5,6,NaN,1
6,7,1.0,1
7,8,1.0,1
8,9,1.0,1
9,10,NaN,1


In [102]:
cardnum = df['CARDNUM'].values.reshape(-1).tolist()
cardno = df_result_predict['cardno'].values.reshape(-1).tolist()

In [103]:
clusters = []
for index1, val1 in enumerate(cardnum):
    found = False
    for index2, val2 in enumerate(cardno):
        if(val1 == val2):
            found = True
            clusters.append(df_result_predict.ix[index2, 'test'])
            break
    if(found == False):
        clusters.append(None)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  import sys


In [104]:
print(clusters)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 5, 1, 1, 4, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 5, 1, 5, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 5, 5, 1, 1, 5, 1, 1, 1, 1, 1, 1, 5, 1, 4, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 5, 1, 1, 1, 4, 1, 1, 1, 1, 4, 1, 1, 1, 1, 4, 1, 1, 5, 1, 5, 5, 1, 1, 1, 5]


In [117]:
unique_elements, counts_elements = np.unique(clusters, return_counts=True)
print("Frequency of unique values of the said array:")
print(np.asarray((unique_elements, counts_elements)))

Frequency of unique values of the said array:
[[ 1  4  5]
 [78  9 13]]


In [105]:
df['cluster'] = clusters

In [106]:
# df.to_csv('mba_cluster.csv', index=False)

In [150]:
df.iloc[24]

CARDNUM                   4
PROD1      1441: BABY\nFOOD
PROD2                   NaN
PROD3                   NaN
PROD4                   NaN
PROD5                   NaN
PROD6                   NaN
PROD7                   NaN
PROD8                   NaN
PROD9                   NaN
PROD10                  NaN
PROD11                  NaN
PROD12                  NaN
PROD13                  NaN
PROD14                  NaN
PROD15                  NaN
PROD16                  NaN
PROD17                  NaN
PROD18                  NaN
PROD19                  NaN
PROD20                  NaN
PROD21                  NaN
PROD22                  NaN
PROD23                  NaN
PROD24                  NaN
PROD25                  NaN
PROD26                  NaN
PROD27                  NaN
PROD28                  NaN
PROD29                  NaN
                 ...       
PROD31                  NaN
PROD32                  NaN
PROD33                  NaN
PROD34                  NaN
PROD35              

In [168]:
print(clusters)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 5, 1, 1, 4, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 5, 1, 5, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 5, 5, 1, 1, 5, 1, 1, 1, 1, 1, 1, 5, 1, 4, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 5, 1, 1, 1, 4, 1, 1, 1, 1, 4, 1, 1, 1, 1, 4, 1, 1, 5, 1, 5, 5, 1, 1, 1, 5]


In [169]:
# grouping cluster

for i in range(0, 10):
    if(i in clusters):
        df_cluster = pd.DataFrame(columns=df.columns)
        for x, value in enumerate(clusters):
            if(value == i):
                print(x)
                df_cluster = df_cluster.append(df.iloc[x], ignore_index = True)

        df_cluster.to_csv('cluster_'+str(i)+'.csv', index=False)

0
1
2
3
4
5
6
7
8
10
11
14
15
16
17
18
19
20
21
22
23
25
26
27
28
30
32
33
34
35
37
38
39
40
41
42
43
46
47
49
50
51
52
53
54
56
58
59
60
61
62
64
65
66
67
68
69
70
72
73
74
76
77
78
80
81
82
83
85
86
87
88
90
91
93
96
97
98
12
24
36
57
63
71
79
84
89
9
13
29
31
44
45
48
55
75
92
94
95
99


In [144]:
df.head()

,CARDNUM,PROD1,PROD2,PROD3,PROD4,PROD5,PROD6,PROD7,PROD8,PROD9,...,PROD51,PROD52,PROD53,PROD54,PROD55,PROD56,PROD57,PROD58,PROD59,cluster
0,3,1504:\nCHEESE,2201:\nSAYUR TAN,1003: FRUIT\nJUI,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,6,1424:\nCHOCOLATE,1008:\nWATER,1500: FRESH\nMIL,1453:\nINSTANT D,1421:\nSOFT\nCAND,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,2,1222: Sanitary,1502: DAIRY\nDES,1414: ROLLS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,5,1440: BABY\nMILK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,10,1440: BABY\nMILK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [85]:
print([1, 1, 1, 1, 1, 1, 1, 1, 1, 5, 1, 1, 4, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 5, 1, 5, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 5, 5, 1, 1, 5, 1, 1, 1, 1, 1, 1, 5, 1, 4, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 5, 1, 1, 1, 4, 1, 1, 1, 1, 4, 1, 1, 1, 1, 4, 1, 1, 5, 1, 5, 5, 1, 1, 1, 5])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 5, 1, 1, 4, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 5, 1, 5, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 5, 5, 1, 1, 5, 1, 1, 1, 1, 1, 1, 5, 1, 4, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 5, 1, 1, 1, 4, 1, 1, 1, 1, 4, 1, 1, 1, 1, 4, 1, 1, 5, 1, 5, 5, 1, 1, 1, 5]


In [ ]:
[3, 6, 2, 5, 10, 8, 3, 8, 8, 1, 8, 3, 4, 1, 8, 6, 3, 5, 9, 8, 3, 5, 7, 6, 4, 8, 3, 9, 7, 1, 2, 1, 8, 5, 9, 2, 4, 10, 5, 6, 2, 6, 5, 3, 1, 1, 7, 5, 1, 7, 8, 10, 7, 8, 3, 1, 6, 4, 9, 10, 9, 5, 3, 4, 7, 8, 10, 8, 9, 3, 10, 4, 2, 2, 5, 1, 8, 10, 3, 4, 2, 8, 7, 5, 4, 3, 7, 6, 7, 4, 2, 7, 1, 6, 1, 1, 6, 10, 8, 1]

In [2]:
df = pd.read_excel('Persona.xlsx', sheet_name='Market Basket')

In [3]:
df.head()

,Transaction_ID,CardNo,Prod1,Prod2,Prod3,Prod4,Prod5
0,1,83,kurma,alpokat,kurma,pisang,sawo
1,2,31,naga merah,naga merah,jambu,Anggur,alpokat
2,3,1,pisang,pir,alpokat,jambu,jambu
3,4,51,sawo,naga merah,Jeruk,sawo,pisang
4,5,85,alpokat,pir,alpokat,mangga,Anggur


In [4]:
df.shape

(199, 7)

In [5]:
df.to_csv('persona_market_basket.csv', index=False)

### read data from csv

##### https://medium.com/@hafizhan.aliady/market-basket-analysis-acossiation-rule-menggunakan-python-1012f9e1611d
##### https://pbpython.com/market-basket-analysis.html

In [7]:
df = pd.read_csv('persona_market_basket.csv')

In [8]:
df.head()

,Transaction_ID,CardNo,Prod1,Prod2,Prod3,Prod4,Prod5
0,1,83,kurma,alpokat,kurma,pisang,sawo
1,2,31,naga merah,naga merah,jambu,Anggur,alpokat
2,3,1,pisang,pir,alpokat,jambu,jambu
3,4,51,sawo,naga merah,Jeruk,sawo,pisang
4,5,85,alpokat,pir,alpokat,mangga,Anggur


In [9]:
df.shape

(199, 7)

### remove transaction_id and CardNo column

In [10]:
df = df.drop(df.columns[[0, 1]], axis=1)

In [11]:
df.head()

,Prod1,Prod2,Prod3,Prod4,Prod5
0,kurma,alpokat,kurma,pisang,sawo
1,naga merah,naga merah,jambu,Anggur,alpokat
2,pisang,pir,alpokat,jambu,jambu
3,sawo,naga merah,Jeruk,sawo,pisang
4,alpokat,pir,alpokat,mangga,Anggur


### check are there any null value

In [12]:
print(df.isna().sum())

Prod1    0
Prod2    0
Prod3    0
Prod4    0
Prod5    0
dtype: int64


In [13]:
# convert pandas dataframe to list
records = []
for i in range(0, df.shape[0]):
    records.append([str(df.values[i,j]) for j in range(0, df.shape[1])])

In [14]:
records

[['kurma', 'alpokat', 'kurma', 'pisang', 'sawo'],
 ['naga merah', 'naga merah', 'jambu', 'Anggur', 'alpokat'],
 ['pisang', 'pir', 'alpokat', 'jambu', 'jambu'],
 ['sawo', 'naga merah', 'Jeruk', 'sawo', 'pisang'],
 ['alpokat', 'pir', 'alpokat', 'mangga', 'Anggur'],
 ['kurma', 'sawo', 'kurma', 'mangga', 'mangga'],
 ['mangga', 'kurma', 'Jeruk', 'alpokat', 'sawo'],
 ['Jeruk', 'Jeruk', 'sawo', 'Jeruk', 'naga merah'],
 ['Jeruk', 'Jeruk', 'mangga', 'naga merah', 'naga merah'],
 ['mangga', 'Anggur', 'jambu', 'alpokat', 'naga merah'],
 ['Jeruk', 'jambu', 'sawo', 'sawo', 'kurma'],
 ['pir', 'mangga', 'pir', 'Anggur', 'naga merah'],
 ['naga merah', 'jambu', 'Jeruk', 'jambu', 'Jeruk'],
 ['pir', 'naga merah', 'alpokat', 'Anggur', 'Anggur'],
 ['Anggur', 'naga merah', 'pir', 'pisang', 'pir'],
 ['pisang', 'alpokat', 'pisang', 'kurma', 'Jeruk'],
 ['mangga', 'Anggur', 'mangga', 'mangga', 'mangga'],
 ['pir', 'jambu', 'sawo', 'mangga', 'kurma'],
 ['naga merah', 'Jeruk', 'kurma', 'naga merah', 'sawo'],
 ['ma

In [16]:
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_ary = te.fit(records).transform(records)
df_transform = pd.DataFrame(te_ary, columns=te.columns_)
df_transform

,Anggur,Jeruk,alpokat,jambu,kurma,mangga,naga merah,pir,pisang,sawo
0,False,False,True,False,True,False,False,False,True,True
1,True,False,True,True,False,False,True,False,False,False
2,False,False,True,True,False,False,False,True,True,False
3,False,True,False,False,False,False,True,False,True,True
4,True,False,True,False,False,True,False,True,False,False
5,False,False,False,False,True,True,False,False,False,True
6,False,True,True,False,True,True,False,False,False,True
7,False,True,False,False,False,False,True,False,False,True
8,False,True,False,False,False,True,True,False,False,False
9,True,False,True,True,False,True,True,False,False,False


In [17]:
df_transform.shape

(199, 10)

In [18]:
df_transform.dtypes

Anggur        bool
Jeruk         bool
alpokat       bool
jambu         bool
kurma         bool
mangga        bool
naga merah    bool
pir           bool
pisang        bool
sawo          bool
dtype: object

In [52]:
# Build up the frequent items# Build  
frequent_itemsets = apriori(df_transform, min_support=0, use_colnames=True)

In [53]:
frequent_itemsets.shape

(1023, 2)

In [33]:
frequent_itemsets.head()

,support,itemsets
0,0.422111,(Anggur)
1,0.396985,(Jeruk)
2,0.402010,(alpokat)
3,0.437186,(jambu)
4,0.412060,(kurma)


In [54]:
many_bundling = 4
for index, row in frequent_itemsets.iterrows():
    if(len(row['itemsets']) == many_bundling):
        result_bundling = frequent_itemsets.loc[[index]]
        break

result_bundling

,support,itemsets
175,0.015075,"(alpokat, jambu, Jeruk, Anggur)"


In [39]:
len(bundling)

120

In [43]:
frequent_itemsets.loc[[174]]

,support,itemsets
174,0.030151,"(pisang, pir, sawo)"


In [44]:
frequent_itemsets.loc[[55]]

,support,itemsets
55,0.045226,"(alpokat, Jeruk, Anggur)"


In [40]:
bundling

[55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174]

In [27]:
frequent_itemsets.to_csv('frequent_itemsets.csv', index=False)

In [28]:
frequent_itemsets

,support,itemsets
0,0.422111,(Anggur)
1,0.396985,(Jeruk)
2,0.402010,(alpokat)
3,0.437186,(jambu)
4,0.412060,(kurma)
5,0.391960,(mangga)
6,0.422111,(naga merah)
7,0.376884,(pir)
8,0.417085,(pisang)
9,0.447236,(sawo)


In [24]:
# Create the rules# Create 
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=0)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Jeruk),(Anggur),0.396985,0.422111,0.130653,0.329114,0.779687,-0.036918,0.861382
1,(Anggur),(Jeruk),0.422111,0.396985,0.130653,0.309524,0.779687,-0.036918,0.873332
2,(alpokat),(Anggur),0.402010,0.422111,0.150754,0.375000,0.888393,-0.018939,0.924623
3,(Anggur),(alpokat),0.422111,0.402010,0.150754,0.357143,0.888393,-0.018939,0.930207
4,(jambu),(Anggur),0.437186,0.422111,0.150754,0.344828,0.816913,-0.033787,0.882042
5,(Anggur),(jambu),0.422111,0.437186,0.150754,0.357143,0.816913,-0.033787,0.875489
6,(kurma),(Anggur),0.412060,0.422111,0.145729,0.353659,0.837834,-0.028206,0.894093
7,(Anggur),(kurma),0.422111,0.412060,0.145729,0.345238,0.837834,-0.028206,0.897944
8,(mangga),(Anggur),0.391960,0.422111,0.150754,0.384615,0.911172,-0.014697,0.939070
9,(Anggur),(mangga),0.422111,0.391960,0.150754,0.357143,0.911172,-0.014697,0.945840


In [51]:
rules.shape

(4, 9)

In [52]:
rules[ (rules['lift'] >= 1) &
       (rules['confidence'] >= 0.4) ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(pisang),(Anggur),0.417085,0.422111,0.190955,0.457831,1.084624,0.014899,1.065885
1,(Anggur),(pisang),0.422111,0.417085,0.190955,0.452381,1.084624,0.014899,1.064453
2,(Jeruk),(kurma),0.396985,0.412060,0.165829,0.417722,1.013739,0.002247,1.009723
3,(kurma),(Jeruk),0.412060,0.396985,0.165829,0.402439,1.013739,0.002247,1.009127


In [54]:
rules.to_csv('rule.csv', index=False)